In [1]:
import sys
sys.path.append("../")

In [2]:
from leadPredictors.model_teams import NeuralNetTeams
from leadPredictors.model_pairs import NeuralNetPairs
from itertools import combinations
import random
import pickle

import os
import torch
import numpy as np

In [3]:
import queue

q = None
q = queue.Queue()

def inpu(read_from_queue=False, timeout=10):
    global q
    if read_from_queue:
        return q.get(timeout=timeout)
    else:
        return input()
        
# q.put('1')
# q.put('3')
# q.put('5')
# q.put('0')
while 1:
    try:
        txt = inpu(read_from_queue=(q != None), timeout=3)
        print(txt)
        if txt == 'q':
            break
    except:
        print("Finished printing")
        break



Finished printing


In [4]:
file_name = "pokedex.txt"
print(file_name, '\n')

start = 0
pokedex = {}
pokedex2 = {}

file = open(file_name, 'r')
file2 = open("pokedex_dict.txt", 'w')
    
Lines = file.readlines()

for i in range(len(Lines)):
    index = Lines[start].find('\t')
    num = Lines[start][index + 2:index + 5]
    idx = Lines[start][index + 6:].find('\t')
    poke_name = Lines[start][index + 6: index + 6 + idx].lower()
    
    file2.write(str(start))
#     file2.write(num)
    file2.write('\n')
    file2.write(poke_name)
    file2.write('\n')
    
    pokedex[poke_name] = start
    pokedex2[start] = poke_name
    
    start += 1
    
file.close()
file2.close()

f = open("../Simulator/poke_to_idx.pkl", 'wb')
pickle.dump(pokedex, f)
f.close()

f = open("../Simulator/idx_to_poke.pkl", 'wb')
pickle.dump(pokedex2, f)
f.close()



pokedex

pokedex.txt 



{'bulbasaur': 0,
 'ivysaur': 1,
 'venusaur': 2,
 'charmander': 3,
 'charmeleon': 4,
 'charizard': 5,
 'squirtle': 6,
 'wartortle': 7,
 'blastoise': 8,
 'caterpie': 9,
 'metapod': 10,
 'butterfree': 11,
 'weedle': 12,
 'kakuna': 13,
 'beedrill': 14,
 'pidgey': 15,
 'pidgeotto': 16,
 'pidgeot': 17,
 'rattata': 18,
 'rattata-alola': 19,
 'raticate': 20,
 'raticate-alola': 21,
 'spearow': 22,
 'fearow': 23,
 'ekans': 24,
 'arbok': 25,
 'pikachu': 26,
 'raichu': 27,
 'raichu-alola': 28,
 'sandshrew': 29,
 'sandshrew-alola': 30,
 'sandslash': 31,
 'sandslash-alola': 32,
 'nidoranâ™€': 33,
 'nidorina': 34,
 'nidoqueen': 35,
 'nidoranâ™‚': 36,
 'nidorino': 37,
 'nidoking': 38,
 'clefairy': 39,
 'clefable': 40,
 'vulpix': 41,
 'vulpix-alola': 42,
 'ninetales': 43,
 'ninetales-alola': 44,
 'jigglypuff': 45,
 'wigglytuff': 46,
 'zubat': 47,
 'golbat': 48,
 'oddish': 49,
 'gloom': 50,
 'vileplume': 51,
 'paras': 52,
 'parasect': 53,
 'venonat': 54,
 'venomoth': 55,
 'diglett': 56,
 'diglett-alola'

In [5]:
%%time

dir_name = "../Game_Profiles/"
team_name = "trick room"
full_path = dir_name + team_name

print(full_path, '\n\n')
data_teams = np.empty((0,979))
labels_teams = []

data_pairs = np.empty((0,979))
labels_pairs = []

da_stats = {}

start = 0
for filename in os.listdir(full_path):
    file = open(full_path + '/' + filename, 'r')
    print(filename)
    Lines = file.readlines()
    
    for i in range(len(Lines)):
        if(Lines[i] == "--------------------\n"):
            start = i + 1
            break
        else:
            continue
            
    poketeam = []        
    
    # Collect all pokemon on enemy team
    while(start < len(Lines)):
        
        if "No more" in Lines[start]:
            break
        
        # Get lead status
        if "lead" in Lines[start].strip():
            status = 2
        elif "back" in Lines[start].strip():
            status = 1
        else:
            status = 0
        
        start += 1 # Go to name
        
        #Get name
        poke_name = Lines[start].strip().lower()

        start += 13 #skip to next pokemon
        
        print('{:14.14s}'.format(poke_name), '\t', pokedex[poke_name], end='\t')
        if status == 2:
            print("lead")
        elif status == 1:
            print("back")
        else:
            print("absent")
        poketeam.append((pokedex[poke_name], status))
    
    
    # Make input and output vector
    team = torch.zeros(979)
    
    lead = []
    back = []
    absent = []
    
    for (poke, stat) in poketeam:
        team[poke] = 1
        if stat == 2:
            lead.append(poke)
        elif stat == 1:
            back.append(poke)
        else:
            absent.append(poke)
    
    data_teams = np.vstack([data_teams, team])
    labels_teams.append((absent, back, lead) )
    
#     print(poketeam, '\n')

    # Make all combinations
    for combo in combinations(poketeam, 2):  
        pair = torch.zeros(979)
        pair[combo[0][0]] = 1
        pair[combo[1][0]] = 1
        print(combo, end='\t')
        if combo[0][1] == 2 and combo[1][1] == 2:
            print("This is the lead", end='')
        
        isLead = int(combo[0][1] == 2 and combo[1][1] == 2)
        
        data_pairs = np.vstack([data_pairs, pair])
        labels_pairs.append(torch.tensor(isLead))
        print()
    print()
    
    for (poke1, s1) in poketeam:
        
        poke1 = pokedex2[poke1]
        
        if poke1 not in da_stats:
            da_stats[poke1] = {}
        for (poke2, s2) in poketeam:
            poke2 = pokedex2[poke2]
            if poke2 in da_stats[poke1]:
                da_stats[poke1][poke2] += 1
            else:
                da_stats[poke1][poke2] = 1
            
            if s1 == 2 and s2 == 2:
                if "lead" not in da_stats[poke1]:
                    da_stats[poke1]["lead"] = {}
                if poke2 not in da_stats[poke1]["lead"]:
                    da_stats[poke1]["lead"][poke2] = 1
                else:
                    da_stats[poke1]["lead"][poke2] += 1
            
            if s1 == 1 and s2 == 1:
                if "back" not in da_stats[poke1]:
                    da_stats[poke1]["back"] = {}
                if poke2 not in da_stats[poke1]["back"]:
                    da_stats[poke1]["back"][poke2] = 1
                else:
                    da_stats[poke1]["back"][poke2] += 1
            
            if s1 == 0 and s2 == 0:
                if "absent" not in da_stats[poke1]:
                    da_stats[poke1]["absent"] = {}
                if poke2 not in da_stats[poke1]["absent"]:
                    da_stats[poke1]["absent"][poke2] = 1
                else:
                    da_stats[poke1]["absent"][poke2] += 1
                
    

labels_pairs = np.array(labels_pairs)

file.close()

../Game_Profiles/trick room 


GameProfile000000
hatterene      	 932	lead
indeedee-f     	 950	absent
liepard        	 562	lead
dusclops       	 391	absent
torkoal        	 356	absent
dhelmise       	 853	absent
((932, 2), (950, 0))	
((932, 2), (562, 2))	This is the lead
((932, 2), (391, 0))	
((932, 2), (356, 0))	
((932, 2), (853, 0))	
((950, 0), (562, 2))	
((950, 0), (391, 0))	
((950, 0), (356, 0))	
((950, 0), (853, 0))	
((562, 2), (391, 0))	
((562, 2), (356, 0))	
((562, 2), (853, 0))	
((391, 0), (356, 0))	
((391, 0), (853, 0))	
((356, 0), (853, 0))	

GameProfile000001
reuniclus      	 638	lead
marowak-alola  	 128	absent
conkeldurr     	 587	back
indeedee-f     	 950	absent
tyranitar      	 278	absent
primarina      	 797	lead
((638, 2), (128, 0))	
((638, 2), (587, 1))	
((638, 2), (950, 0))	
((638, 2), (278, 0))	
((638, 2), (797, 2))	This is the lead
((128, 0), (587, 1))	
((128, 0), (950, 0))	
((128, 0), (278, 0))	
((128, 0), (797, 2))	
((587, 1), (950, 0))	
((587, 1), (278, 0))	
((

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if (torch.cuda.is_available()):
  print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()
device

GeForce GTX 1660 SUPER


device(type='cuda')

In [7]:
test_size = 0

copy_data = np.copy(data_teams)
copy_labels = labels_teams.copy()

index = np.random.choice(len(copy_data), size=test_size, replace=False).tolist()
index.sort()
index = index[::-1]

test_data = copy_data[index]
test_labels = [copy_labels[x] for x in index]

train_data = np.delete(copy_data, index, axis=0)
train_labels = labels_teams.copy()
for x in index:
    train_labels.pop(x)

print(data_teams.shape)
print(test_data.shape)
print(train_data.shape)

test_data = torch.from_numpy(test_data).to(device)
# test_labels = torch.from_numpy(test_labels).to(device).view(-1, 1)

train_data = torch.from_numpy(train_data).to(device)
# train_labels = torch.from_numpy(train_labels).to(device).view(-1, 1)

(440, 979)
(0, 979)
(440, 979)


In [8]:
def fitTeams(net=None, num_epoch=10, batch_size=10):
    """ Fit a neural net.  Use the full batch size.
    
    @param batch_size: The size of each batch to train on.
    @param num_epoch: The number of epochs to train on
    @param net: A NeuralNet object
    
    @return losses: losses for each batch
    @return net: A NeuralNet object
    """
    
    # Defining the neural network

    if (net == None):
        net = NeuralNetTeams(0.01)
    net.to(device)
    
    # Training

    losses = []
    for epoch in range(num_epoch):
        # Training
        i = 0
        print("epoch:", epoch)
        while (i + 1) * batch_size < len(train_data) + 1:
        
            batch = train_data[i * batch_size: (i + 1) * batch_size]
            target = train_labels[i * batch_size: (i + 1) * batch_size]
        
            batch = batch.float().to(device)
                    
            loss = net.step(batch, target)
            losses.append(loss)
            print('Loss', i, ':', "{:.5f}".format(loss), end='\r')

            i += 1
        print()
        
    return losses, net

In [9]:
%%time
losses, mynet = fitTeams(num_epoch=100)
mynet.save_model('sample_model')

epoch: 0
Loss 43 : 38.75063
epoch: 1
Loss 43 : 38.52121
epoch: 2
Loss 43 : 38.37505
epoch: 3
Loss 43 : 38.2721335 : 38.27102
epoch: 4
Loss 43 : 38.08772
epoch: 5
Loss 43 : 37.82357
epoch: 6
Loss 43 : 37.4494932 : 36.91643
epoch: 7
Loss 43 : 37.01591
epoch: 8
Loss 43 : 36.51993
epoch: 9
Loss 43 : 35.97986
epoch: 10
Loss 43 : 35.39962
epoch: 11
Loss 43 : 34.80720: 32.92891
epoch: 12
Loss 43 : 34.21194
epoch: 13
Loss 43 : 33.6227028 : 31.43525
epoch: 14
Loss 43 : 32.99560
epoch: 15
Loss 43 : 32.38494
epoch: 16
Loss 43 : 31.78373
epoch: 17
Loss 43 : 31.1495226 : 27.62334
epoch: 18
Loss 43 : 30.54466
epoch: 19
Loss 43 : 29.90984
epoch: 20
Loss 43 : 29.26337
epoch: 21
Loss 43 : 28.62063
epoch: 22
Loss 43 : 28.03703
epoch: 23
Loss 43 : 27.38869
epoch: 24
Loss 43 : 26.78707
epoch: 25
Loss 43 : 26.2341323 : 21.80507
epoch: 26
Loss 43 : 25.64808
epoch: 27
Loss 43 : 25.07646
epoch: 28
Loss 43 : 24.54134
epoch: 29
Loss 43 : 24.0024020.46474
epoch: 30
Loss 43 : 23.47421
epoch: 31
Loss 43 : 22.96984

In [10]:
x = random.randint(0, len(data_teams))
print(x)
output = mynet(torch.from_numpy(data_teams[x]).float().to(device))

print(output[0, labels_teams[x][0]])
print(output[0, labels_teams[x][1]])
print(output[0, labels_teams[x][2]])

print(labels_teams[x])
for i in labels_teams[x]:
    for j in i:
        print('{:8.12s}'.format(pokedex2[j]), '\t', j)

353
tensor([[0.8819, 0.0034, 0.1148],
        [0.9624, 0.0082, 0.0295]], device='cuda:0', grad_fn=<IndexBackward>)
tensor([[0.0503, 0.9109, 0.0389],
        [0.0066, 0.9489, 0.0445]], device='cuda:0', grad_fn=<IndexBackward>)
tensor([[1.9971e-04, 3.0083e-02, 9.6972e-01],
        [4.1338e-02, 1.4321e-02, 9.4434e-01]], device='cuda:0',
       grad_fn=<IndexBackward>)
([884, 66], [2, 356], [650, 670])
rillaboom 	 884
golduck  	 66
venusaur 	 2
torkoal  	 356
amoonguss 	 650
chandelure 	 670


In [13]:
team = torch.zeros(979)
indices = [957, 583, 857, 5, 391, 659]
for i in indices:
    team[i] = 1

output = mynet(team.float().to(device))

print("pokemon\t\tabsent\treserve\tlead")
print('--------------------------------------')

for i in indices:
    print('{:4.12s}'.format(pokedex2[i]), end='\t')
    for o in output[0, i].tolist():
        print('{:.4f}'.format(o), end='\t')
    print()


lead_scores = []


for i in indices:
    lead_scores.append((i, output[0, i] ))
    
lead_scores.sort(key=lambda x:-x[1][2])
print()
print(pokedex2[lead_scores[0][0]], pokedex2[lead_scores[1][0]])

pokemon		absent	reserve	lead
--------------------------------------
dracovish	0.7104	0.0229	0.2667	
excadrill	0.9817	0.0084	0.0099	
tapu koko	0.2543	0.2489	0.4968	
charizard	0.2194	0.6267	0.1539	
dusclops	0.5860	0.3702	0.0438	
ferrothorn	0.3358	0.6479	0.0163	

tapu koko dracovish


In [14]:
test_size = 0

copy_data = np.copy(data_pairs)
copy_labels = np.copy(labels_pairs)

index = np.random.choice(len(copy_data), size=test_size, replace=False).tolist()

test_data = copy_data[index]
test_labels = copy_labels[index]

train_data = np.delete(copy_data, index, axis=0)
train_labels = np.delete(copy_labels, index, axis=0)

print(data_pairs.shape)
print(test_data.shape)
print(train_data.shape)
print(train_labels.shape)

test_data = torch.from_numpy(test_data).to(device)
test_labels = torch.from_numpy(test_labels).to(device).view(-1, 1)

train_data = torch.from_numpy(train_data).to(device)
train_labels = torch.from_numpy(train_labels).to(device).view(-1, 1)

(6582, 979)
(0, 979)
(6582, 979)
(6582,)


In [18]:
def fitPairs(net=None, num_epoch=100, batch_size=10):
    """ Fit a neural net.  Use the full batch size.
    
    @param batch_size: The size of each batch to train on.
    @param num_epoch: The number of epochs to train on
    @param net: A NeuralNet object
    
    @return losses: losses for each batch
    @return net: A NeuralNet object
    """
    
    # Defining the neural network

    if (net == None):
        net = NeuralNetPairs(0.0005)
    net.to(device)
    
    # Training

    losses = []
    for epoch in range(num_epoch):
        # Training
        i = 0
        print("epoch:", epoch)
        while (i + 1) * batch_size < len(train_data) + 1:
        
            batch = train_data[i * batch_size: (i + 1) * batch_size]
            target = train_labels[i * batch_size: (i + 1) * batch_size]
        
            batch = batch.float().to(device)
            target = target.float().to(device)
                    
            loss = net.step(batch, target)
            losses.append(loss)
            if i % 25 == 0:
                print('Loss', i, ':', "{:.5f}".format(loss), end='\r')

            i += 1
        print()
        
    return losses, net

In [19]:
%%time
losses, mynet2 = fitPairs()

epoch: 0
Loss 650 : 1.05056
epoch: 1
Loss 650 : 0.566590.54902
epoch: 2
Loss 650 : 0.34167250 : 0.43899
epoch: 3
Loss 650 : 0.240150.26945
epoch: 4
Loss 650 : 0.18330
epoch: 5
Loss 650 : 0.14817425 : 0.16407
epoch: 6
Loss 650 : 0.12483
epoch: 7
Loss 650 : 0.10845
epoch: 8
Loss 650 : 0.09646: 0.90774
epoch: 9
Loss 650 : 0.08739
epoch: 10
Loss 650 : 0.08034
epoch: 11
Loss 650 : 0.07473525 : 0.89498
epoch: 12
Loss 650 : 0.07020
epoch: 13
Loss 650 : 0.06648525 : 0.89612
epoch: 14
Loss 625 : 0.06692: 0.06450650 : 0.06339
epoch: 15
Loss 650 : 0.06079
epoch: 16
Loss 650 : 0.05858400 : 0.05941
epoch: 17
Loss 650 : 0.05670
epoch: 18
Loss 650 : 0.05508
epoch: 19
Loss 650 : 0.05367
epoch: 20
Loss 650 : 0.05245: 0.05927350 : 0.90605
epoch: 21
Loss 650 : 0.05138450 : 0.89851
epoch: 22
Loss 650 : 0.05045
epoch: 23
Loss 650 : 0.04962
epoch: 24
Loss 650 : 0.04889
epoch: 25
Loss 650 : 0.04825
epoch: 26
Loss 650 : 0.0476700 : 0.054140.05061
epoch: 27
Loss 650 : 0.04716
epoch: 28
Loss 650 : 0.0467100 : 0

In [21]:
x = random.randint(0, len(data_pairs) - 15)
print(x, '\n')

output = mynet2(torch.from_numpy(data_pairs[x:x+15]).float().to(device))
answers = labels_pairs[x:x+15] 

print("Predicted\tActual")
for i in range(len(answers)):
    print('{:.5f}\t\t{:d}'.format(output[i].item(), answers[i]))

5245 

Predicted	Actual
0.07564		0
0.07446		0
0.07606		0
0.06749		0
0.07403		0
0.07354		1
0.06518		0
0.07175		0
0.06679		0
0.07326		0
0.06489		0
0.07566		0
0.06766		0
0.06202		0
0.07218		0
